In [ ]:
import os 

# Run global_variables, synthesize_functions and mlp_model
%run ~/violin-renderer/src/global_variables.ipynb
%run ~/violin-renderer/src/models/mlp/mlp_model.ipynb
%run ~/violin-renderer/src/synthesize/synthesize_functions.ipynb

In [ ]:
# Create an output directory in /src, generate all results from the test dataset and synthesize all of them
# Setting up paths and output folders
SOURCE_INPUT_PATH =  HOME_PATH + '/bach-violin-dataset/dataset/source-input'
TEMPO_PATH =  HOME_PATH + '/bach-violin-dataset/dataset/tempos'
OUTPUT_PATH = HOME_PATH + '/src/mlp-output'
composer_directories = os.listdir(SOURCE_INPUT_PATH)

# Initialize models
mlp_model = MLP(input_size=3, hidden_size=HIDDEN_LAYER_SIZE, output_size=2)

# Uncomment one of the models
mlp_model.load_state_dict(torch.load(HOME_PATH + '/src/models/mlp/states/mlp_model.pt'))
# mlp_model.load_state_dict(torch.load(HOME_PATH + '/src/models/mlp/states/mlp_model_2.pt'))

# Get generated output values from all songs
results = generate_all_testing_data(mlp_model)
# print(results)

# Get MSE values
MSE = [float(num) for num in MSE_error(mlp_model)]
print('MSE values:', MSE)
print('MSE Min: %.3f' % min(MSE))
print('MSE Max: %.3f' % max(MSE))
print('MSE Mean: %.3f' % np.mean(MSE))
print('MSE Standard Deviation: %.3f' % np.std(MSE))

In [ ]:
# Micro Average
testing_source_input, _ = processed_testing_datasets()
number_of_notes = [len(song) for song in testing_source_input.values()]
weighted_MSE = [mse_error * num_of_notes / sum(number_of_notes) for mse_error, num_of_notes in zip(MSE, number_of_notes)]
print("Micro MSE: %.2f" % sum(weighted_MSE))
print("Micro Root MSE: %.2f" % np.sqrt(sum(weighted_MSE)))

In [ ]:
# Synthesize
if not os.path.isdir(OUTPUT_PATH):
    os.mkdir(OUTPUT_PATH)

for notes_path, generated_results in results.items():
    name = '/' + notes_path.split('/')[-1]
    name = name.replace('.csv', '')
    generated_timings = [(note[0], note[1]) for note in generated_results]
    synthesize_generated_output(notes_path, generated_timings, OUTPUT_PATH, name)